In [1]:
from hipars import Sorting
import random
import numpy as np

total_side_length = 40
filled_length = 26
state_array = np.ndarray((total_side_length,total_side_length),dtype=bool)

for i in range(total_side_length):
    for j in range(total_side_length):
        state_array[i,j] = random.random() > 0.5
filled_shape = np.array([filled_length,filled_length],np.int32)

row_start = (total_side_length - filled_length) // 2
col_start = (total_side_length - filled_length) // 2
comp_zone = (row_start, row_start + filled_length, col_start, col_start + filled_length)

state_array_copy = state_array.copy()
state_array_copy2 = state_array.copy()

sorter = Sorting.Sorting()
sorter.configure_log(log_file_name="test.log")
# Sequential sorting
sequential_moves = sorter.sort_sequentially(state_array_copy, (row_start, row_start + filled_length), (col_start, col_start + filled_length))
if sequential_moves is not None:
    for move in sequential_moves:
        print("Move contains " + str(len(move.sites_list) - 1) + " steps with a total length of " + str(move.distance))
else:
    print("No sequential sequence found")
sorter.flush_logs()

# Parallel sorting
parallel_moves = sorter.sort_parallel(state_array_copy2, (row_start, row_start + filled_length), (col_start, col_start + filled_length))
if parallel_moves is not None:
    for move in parallel_moves:
        print("Move shifts " + str(len(move.steps[0].rowSelection)) + " rows and " + \
            str(len(move.steps[0].colSelection)) + " columns for " + str(len(move.steps) - 1) + " steps")
sorter.flush_logs()

Move contains 1 steps with a total length of 1.0
Move contains 1 steps with a total length of 4.0
Move contains 1 steps with a total length of 1.0
Move contains 1 steps with a total length of 1.0
Move contains 1 steps with a total length of 2.0
Move contains 1 steps with a total length of 3.0
Move contains 1 steps with a total length of 5.0
Move contains 1 steps with a total length of 5.0
Move contains 1 steps with a total length of 6.0
Move contains 1 steps with a total length of 3.0
Move contains 1 steps with a total length of 1.0
Move contains 1 steps with a total length of 2.0
Move contains 1 steps with a total length of 1.0
Move contains 1 steps with a total length of 1.0
Move contains 1 steps with a total length of 3.0
Move contains 1 steps with a total length of 3.0
Move contains 1 steps with a total length of 1.0
Move contains 1 steps with a total length of 2.0
Move contains 1 steps with a total length of 2.0
Move contains 1 steps with a total length of 2.0
Move contains 1 step

In [ ]:
import numpy as np
import cv2
from ipywidgets import IntProgress
from IPython.display import display

spacing = 20
radius = 4
size = np.array(state_array.shape) * spacing
channels = 3

row_arange = np.arange(size[0])
col_arange = np.arange(size[1])

def image_coords(row,col):
    image_row = int(spacing / 2 + spacing * row)
    image_col = int(spacing / 2 + spacing * col)
    inside = (row_arange[:,None] - image_row) ** 2 + (col_arange[None, :] - image_col) ** 2 <= (radius ** 2)
    return inside

duration = 2
fps = 60

pause_between_moves = 10
frames_per_site = 2
total_frames = 2 * fps + pause_between_moves
for move in sequential_moves:
    total_frames += move.distance * frames_per_site + pause_between_moves
total_frames = int(np.ceil(total_frames))

f = IntProgress(min=0, max=total_frames)
display(f)

out = cv2.VideoWriter('sequentialMovement.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (size[1], size[0]))
data = np.full((*size,3), 255, dtype='uint8')
for row in range(total_side_length):
    for col in range(total_side_length):
        if state_array[row,col]:
            data[image_coords(row,col),...] = 0
frame_in_move = 0
active_move = None
move_index = 0

for frame in range(total_frames):
    # If no active move: Pause and then select next move
    if active_move is None and frame_in_move >= pause_between_moves and move_index < len(sequential_moves):
        active_move = sequential_moves[move_index]
        data[image_coords(*active_move.sites_list[0]),...] = 255
        frame_in_move = 0
    move_data = data.copy()
    if active_move is not None:
        remaining_frames = frame_in_move
        last_site = active_move.sites_list[0]
        still_in_move = False
        for site in active_move.sites_list[1:]:
            frames_in_submove = np.linalg.norm(np.array(site) - np.array(last_site)) * frames_per_site
            if remaining_frames < frames_in_submove:
                location = np.array(last_site) + (np.array(site) - np.array(last_site)) * remaining_frames / frames_in_submove
                move_data[image_coords(*location),0:2] = 0
                still_in_move = True
                break
            else:
                remaining_frames -= frames_in_submove
            last_site = site
        if not still_in_move:
            frame_in_move = 0
            data[image_coords(*active_move.sites_list[-1]),...] = 0
            active_move = None
            move_index += 1
    frame_in_move += 1
    out.write(move_data)
    f.value += 1
out.release()

IntProgress(value=0, max=13726)

In [ ]:
import numpy as np
import cv2
from ipywidgets import IntProgress
from IPython.display import display

spacing = 20
radius = 4
size = np.array(state_array.shape) * spacing
channels = 3

row_arange = np.arange(size[0])
col_arange = np.arange(size[1])

def image_coords(row,col):
    image_row = int(spacing / 2 + spacing * row)
    image_col = int(spacing / 2 + spacing * col)
    inside = (row_arange[:,None] - image_row) ** 2 + (col_arange[None, :] - image_col) ** 2 <= (radius ** 2)
    return inside

duration = 2
fps = 30

pause_between_moves = 10
pause_between_submoves = 5
frames_per_site = 3
total_frames = 2 * fps
for move in parallel_moves:
    last_step = move.steps[0]
    for step in move.steps[1:]:
        max_col_dist = np.max(np.abs(np.array(step.colSelection) - np.array(last_step.colSelection)))
        max_row_dist = np.max(np.abs(np.array(step.rowSelection) - np.array(last_step.rowSelection)))
        max_dist = np.max([max_col_dist,max_row_dist])
        total_frames += max_dist * frames_per_site + pause_between_submoves
        last_step = step
    total_frames += pause_between_moves - pause_between_submoves
total_frames = int(np.ceil(total_frames))

f = IntProgress(min=0, max=total_frames)
display(f)

out = cv2.VideoWriter('parallelMovement.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (size[1], size[0]))
data = np.full((*size,3), 255, dtype='uint8')
for row in range(total_side_length):
    for col in range(total_side_length):
        if state_array[row,col]:
            data[image_coords(row,col),...] = 0
frame_in_move = 0
active_move = None
move_index = 0

state_array_movement_copy = state_array.copy()

for frame in range(total_frames):
    # If no active move: Pause and then select next move
    if active_move is None and frame_in_move >= pause_between_moves and move_index < len(parallel_moves):
        active_move = parallel_moves[move_index]
        atom_movement_matrix = np.ndarray((len(active_move.steps[0].rowSelection),len(active_move.steps[0].colSelection)), bool)
        for ri,row in enumerate(active_move.steps[0].rowSelection):
            for ci,col in enumerate(active_move.steps[0].colSelection):
                if state_array_movement_copy[int(row + 0.25),int(col + 0.25)]:
                    atom_movement_matrix[ri,ci] = True
                    data[image_coords(row,col),...] = 255
                    state_array_movement_copy[int(row + 0.25),int(col + 0.25)] = False
                else:
                    atom_movement_matrix[ri,ci] = False
        frame_in_move = 0
    move_data = data.copy()
    if active_move is not None:
        remaining_frames = frame_in_move
        last_step = active_move.steps[0]
        still_in_move = False
        for i,step in enumerate(active_move.steps[1:]):
            max_row_dist = np.max(np.abs(np.array(step.rowSelection) - np.array(last_step.rowSelection)))
            max_col_dist = np.max(np.abs(np.array(step.colSelection) - np.array(last_step.colSelection)))
            max_dist = np.max([max_col_dist,max_row_dist])
            frames_in_submove = max_dist * frames_per_site
            if remaining_frames < frames_in_submove:
                move_fraction = remaining_frames / frames_in_submove
                rows = np.array(last_step.rowSelection) + (np.array(step.rowSelection) - np.array(last_step.rowSelection)) * move_fraction
                cols = np.array(last_step.colSelection) + (np.array(step.colSelection) - np.array(last_step.colSelection)) * move_fraction
                for ri,row in enumerate(rows):
                    for ci,col in enumerate(cols):
                        if atom_movement_matrix[ri,ci]:
                            move_data[image_coords(row,col),0:2] = 0
                break
            else:
                remaining_frames -= frames_in_submove
                if remaining_frames < pause_between_submoves:
                    if i < len(active_move.steps) - 2:
                        for ri,row in enumerate(step.rowSelection):
                            for ci,col in enumerate(step.colSelection):
                                if atom_movement_matrix[ri,ci]:
                                    move_data[image_coords(row,col),0:2] = 0
                        break
                    else:
                        frame_in_move = 0
                        for ri,row in enumerate(active_move.steps[-1].rowSelection):
                            for ci,col in enumerate(active_move.steps[-1].colSelection):
                                if atom_movement_matrix[ri,ci]:
                                    data[image_coords(row,col),...] = 0
                                    state_array_movement_copy[int(row + 0.25),int(col + 0.25)] = True
                        move_data = data.copy()
                        active_move = None
                        move_index += 1
                else:
                    remaining_frames -= pause_between_submoves
            last_step = step
    frame_in_move += 1
    out.write(move_data)
    f.value += 1
out.release()

IntProgress(value=0, max=2640)